In [140]:
import pandas as pd
import numpy as np
import datetime, random

In [218]:
df = pd.read_csv('inpatient 2014-2018 creatinine.csv') #1441707, 4
df.pat_mrn_id = df.pat_mrn_id.str.strip('MR').astype('int')
df.time = pd.to_datetime(df.time)
print(df.dtypes)
df.set_index(['pat_mrn_id', 'pat_enc_csn_id'], inplace=True)
df.head(10)

pat_mrn_id                 int32
pat_enc_csn_id             int64
time              datetime64[ns]
creatinine               float64
dtype: object


time  creatinine
pat_mrn_id pat_enc_csn_id                                
1000041    115884935      2015-06-14 20:59:00        1.60
           115884935      2015-06-15 07:54:00        0.90
           115884935      2015-06-16 07:02:00        0.90
           115884935      2015-06-17 07:11:00        0.90
           117378943      2015-07-18 08:39:00        1.10
           117378943      2015-07-19 08:43:00        1.00
           117378943      2015-07-20 08:35:00        0.90
           117378943      2015-07-21 07:48:00        0.80
           147850097      2017-03-20 16:00:00        0.97
           147850097      2017-03-21 02:06:00        1.10

In [142]:
#Quick question to ask: say a patient comes in on a separate encounter within 7 days
#and their creatinine has gone up by 50% since their previous encounter. Does that count as AKI?

#((df.groupby(['pat_mrn_id']).shift(-1) - df).time < datetime.timedelta(days=7)).sum()
#((df.groupby(['pat_mrn_id', 'pat_enc_csn_id']).shift(-1) - df).time < datetime.timedelta(days=7)).sum()

delta_df = df.groupby(['pat_mrn_id', 'pat_enc_csn_id']).shift(-1) - df
delta_df.head(10)

time  creatinine
pat_mrn_id pat_enc_csn_id                            
MR1000041  115884935      0 days 10:55:00       -0.70
           115884935      0 days 23:08:00        0.00
           115884935      1 days 00:09:00        0.00
           115884935                  NaT         NaN
           117378943      1 days 00:04:00       -0.10
           117378943      0 days 23:52:00       -0.10
           117378943      0 days 23:13:00       -0.10
           117378943                  NaT         NaN
           147850097      0 days 10:06:00        0.13
           147850097      1 days 05:32:00        0.10

In [146]:
df['delta_creat'] = delta_df['creatinine']
df['delta_time'] = delta_df['time']
#df[['delta_time, delta_creat']] = delta_df[['time', 'creatinine']]
df.delta_creat = df.delta_creat.shift(1)
df.delta_time = df.delta_time.shift(1)
df.head(10)

time  creatinine  delta_creat  \
pat_mrn_id pat_enc_csn_id                                                
MR1000041  115884935      2015-06-14 20:59:00        1.60          NaN   
           115884935      2015-06-15 07:54:00        0.90        -0.70   
           115884935      2015-06-16 07:02:00        0.90         0.00   
           115884935      2015-06-17 07:11:00        0.90         0.00   
           117378943      2015-07-18 08:39:00        1.10          NaN   
           117378943      2015-07-19 08:43:00        1.00        -0.10   
           117378943      2015-07-20 08:35:00        0.90        -0.10   
           117378943      2015-07-21 07:48:00        0.80        -0.10   
           147850097      2017-03-20 16:00:00        0.97          NaN   
           147850097      2017-03-21 02:06:00        1.10         0.13   

                               delta_time  
pat_mrn_id pat_enc_csn_id                  
MR1000041  115884935                  NaT  
           115884935      0 days 10:55:00  
           115884935      0 days 23:08:00  
           115884935      1 days 00:09:00  
           117378943                  NaT  
           117378943      1 days 00:04:00  
           117378943      0 days 23:52:00  
           117378943      0 days 23:13:00  
           147850097                  NaT  
           147850097      0 days 10:06:00

In [191]:
(df.delta_creat > df.creatinine.shift(1)*0.5)

pat_mrn_id  pat_enc_csn_id
MR1000041   115884935         False
            115884935         False
            115884935         False
            115884935         False
            117378943         False
            117378943         False
            117378943         False
            117378943         False
            147850097         False
            147850097         False
            147850097         False
            147850097         False
MR1000056   111848406         False
            111848406         False
            112813525         False
            112813525         False
            112813525         False
            112813525         False
            114295555         False
            114295555         False
            114295555         False
            114295555         False
MR1000057   104466981         False
            104466981         False
            104466981         False
            104466981         False
            104466981         False
M

In [203]:
condition1 = np.logical_and(df.delta_creat > 0.3, df.delta_time < datetime.timedelta(hours=48))
condition2 = np.logical_and(df.delta_creat > df.creatinine.shift(1)*0.5, df.delta_time < datetime.timedelta(days=7))
df['AKI'] = np.logical_or(condition1, condition2)

#Question: Once you've got AKI you've got AKI for the rest of the encounter, right?
# df.loc['MR1000136']

#Condition 1 examples:
# MR1000136, MR1000468, MR1000507

#Condition 2 examples:
#MR1001546, MR1001806, 

#df[np.logical_xor(condition1, condition2)] gives unique examples of C1 or C2

In [204]:
 df.loc['MR1000']

,time,creatinine,delta_creat,delta_time,AKI
pat_enc_csn_id,,,,,
95064759,2014-01-05 16:11:00,0.9,NaN,NaT,False
95064759,2014-01-06 05:32:00,0.8,-0.1,0 days 13:21:00,False
95064759,2014-01-07 03:19:00,0.8,0.0,0 days 21:47:00,False
95359271,2014-01-13 10:08:00,0.8,NaN,NaT,False
95359271,2014-01-13 10:17:00,0.7,-0.1,0 days 00:09:00,False
95359271,2014-01-14 07:43:00,0.9,0.2,0 days 21:26:00,False
96162241,2014-02-03 22:43:00,0.8,NaN,NaT,False
96162241,2014-02-03 23:07:00,0.8,0.0,0 days 00:24:00,False
96162241,2014-02-05 07:08:00,1.1,0.3,1 days 08:01:00,True


In [171]:
df.index[0]

('MR1000041', 115884935)